In [10]:
pip install scipy


[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: python3.12 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Context

Kirk's approximation is an analytical formula for valuing a call option on a spread between two assets (in this case: UK & FR day-ahead prices). In our context:
- S_1 = UK Forward Price
- S_2 = FR Forward Price
- Sigma_1, Sigma_2 = volatilities of UK and FR
- p (rho) = correlation between them
- K = strike price (capacity cost, often 0 in our case)
- T = time to expiry in years

A key benefit is that Kirk's analytical approximation avoids the computational cost of Monte Carlo simulation while providing accurate spread option values

The output will be EUR per MWh option value, which you can then multiply by capacity to get a notional market value for transmission rights.


In [11]:
import numpy as np
import pandas as pd
from scipy.stats import norm
import sys
import os
sys.path.append(os.path.abspath("/Users/Lyndon.Odia/Desktop/lo-devx/power-spread-option-pricing-main"))
from config import raw_data_dir, processed_data_dir, API_KEY, FR_DOMAIN, START_DATE, END_DATE, FX_GBP_EUR

In [12]:
# Load merged hourly dataset 
data_path = os.path.join(processed_data_dir, "UK_FR_day_ahead_hourly_merged_spread.csv")
df = pd.read_csv(data_path, parse_dates=["datetime"])

In [13]:
# Ensure sorted and clean
df = df.sort_values("datetime").reset_index(drop=True)


In [14]:
S1 = df["UK_price_eur"].mean()  # Mean UK price in EUR/MWh
S2 = df["FR_price"].mean()      # Mean FR price in EUR/MWh

# From return-based volatilities (already annualised in your EDA) - normalise 
sigma1 = 16.08 / S1 #EUR/MWh
sigma2 = 68.60 / S2 #EUR/MWh
rho = 0.142   # Pearson correlation from returns

In [15]:
print(S1, S2)
print(sigma1)
print(sigma2)
print(rho)

81.10352334017234 68.06369511694707
0.19826512262057577
1.00787945588513
0.142


In [16]:
K = 0.76 # Strike price (capacity cost) in EUR/MWh
T = 1/12     # 1 month to expiry

In [17]:
# Kirk's Approximation (#source used: Kirk Formula and Modified Kirk Formula for Spread Option Pricing in Python - Statistics and Risk Modelling - YT)
b = S2 / (S2 + K)
sigma_k = np.sqrt(sigma1**2 - 2*b*rho*sigma1*sigma2 + (b**2)*(sigma2**2))
d1 = (np.log(S1 / (S2 + K)) + 0.5 * sigma_k**2 * T) / (sigma_k * np.sqrt(T))
d2 = d1 - sigma_k * np.sqrt(T)
C = S1 * norm.cdf(d1) - (S2 + K) * norm.cdf(d2) #Where N - (norm.cdf) is the standard normal CDF, C is the price of the spread option

In [18]:
print(f"UK mean price (S1): {S1:.2f} EUR/MWh")
print(f"FR mean price (S2): {S2:.2f} EUR/MWh")
print(f"UK vol (annualised): {sigma1:.2f}")
print(f"FR vol (annualised): {sigma2:.2f}")
print(f"Correlation: {rho:.3f}")
print(f"Spread Option Value (1M, Kirk's): {C:.2f} EUR/MWh")

UK mean price (S1): 81.10 EUR/MWh
FR mean price (S2): 68.06 EUR/MWh
UK vol (annualised): 0.20
FR vol (annualised): 1.01
Correlation: 0.142
Spread Option Value (1M, Kirk's): 16.00 EUR/MWh


### Notes & Intepretations
**Volatility units**:
The annualised volatilities (sigma_1 and sigma_2) are dimensionless hence they represent % variability per year, not a price level. For example, sigma_1 = 0.2 means the UK power price flunctuates 20% per year around its mean level of 81 EUR/MWh

**Why divide S1 and S2**:
kIRK'S formula requires relative volatility, so the sigma values used in the pricing equation are scaled by the mean prices (sigma / S). This converts absolute volatility (EUR/MWh) into proportional volatility, consistent with the lognormal assumption of the model.

**Correlation (rho - 0.142)**:
Indicates weak positive co-movement between UK and FR day-ahead prices - spread optionality remains valuable because prices often move independently.

**Capacity Cost (K)**:
The average auction clearing price for transmission capacity from GB - > FR (IFA2) between Jan–Jul 2025 was approximately €0.76/MWh, based on publicly available data from the Joint Allocation Office (JAO) monthly explicit auctions. This represents the cost paid by market participants to secure physical transmission rights over that interconnector, and serves as the strike price K in our spread option valuation.
Source: https://www.jao.eu/auctions#/

For illustration, the analysis uses the GB - > FR direction. In practice, interconnector capacity is auctioned separately for each direction, and both flows (FR → GB and GB → FR) have distinct prices reflecting directional congestion and market fundamentals

**Interpretation of Result**:
With a one-month expiry and a clearing price / capacity cost K = €0.76/MWh (based on the 2025 JAO monthly auction results for the IFA2 GB-FR border), the Kirk model gives a fair option value of €16.00/MWh.
This can be interpreted as the implied premium per MWh of interconnector transmission capacity between the UK and France for that horizon

**In Practice / Real world extension note**:
In production, each delivery month would use its own JAO clearing price as the strike K_m, and re-estimate volatility, correlation, and means using that month’s data.
For this case study, a single representative value (€0.76/MWh) is used to provide a clean and realistic benchmark.